In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import os

In [2]:
p1 = os.path.join("raw","GTA.csv")
df = pd.read_csv(p1)
df2 = pd.read_csv("raw/causes.csv")

In [3]:
df_final = df[["Year","Glob","NHem","SHem"]]
# Baseline 1951-1980 (14°C) used for the GTA file, baseline for anomalies 1880-1910:
mean_base_df = df_final[(df_final['Year']>1879)&(df_final['Year']<1901)]
mean_base_df["temp"] = mean_base_df['Glob'] + 14
mean_baseline = mean_base_df["temp"].mean()
print(mean_baseline)
df_final['Glob'] = df_final['Glob']+ 0.2

13.79


In [5]:
New_df2 = df2[['All forcings','Human','Natural',
    'Anthropogenic tropospheric aerosol','Greenhouse gases',
    'Land use','Orbital changes','Ozone','Solar','Volcanic']]

for col in New_df2.columns:
    New_df2[f'{col}'] = New_df2[f'{col}'] - float(273.15)
    New_df2[f'{col}'] = New_df2[f'{col}'] - 14.34
New_df2.tail(10)

,All forcings,Human,Natural,Anthropogenic tropospheric aerosol,Greenhouse gases,Land use,Orbital changes,Ozone,Solar,Volcanic
146,0.614646,0.652811,-0.065624,-0.475731,0.947629,-0.128402,0.007362,0.104765,0.017531,-0.018413
147,0.647882,0.662333,-0.024018,-0.465583,0.981132,-0.150241,-0.107271,0.029719,-0.041251,0.003356
148,0.740968,0.702007,-0.022988,-0.475270,0.989026,-0.171568,-0.116840,0.040501,0.002263,0.038056
149,0.778828,0.716130,0.025731,-0.451686,0.978662,-0.185987,-0.059780,0.059232,0.012872,0.033221
150,0.818978,0.693264,0.026963,-0.466707,1.019794,-0.157066,-0.052279,0.016140,-0.018852,0.007629
151,0.830929,0.694239,0.036614,-0.521246,1.062342,-0.166334,-0.100424,0.028246,-0.017479,0.009175
152,0.845254,0.728673,0.055374,-0.467304,1.104759,-0.128149,-0.090166,0.055542,-0.012858,0.014434
153,0.858285,0.769768,0.044229,-0.456452,1.105026,-0.144010,-0.029302,0.013628,-0.010081,0.025221
154,0.892879,0.779479,0.069768,-0.469128,1.125446,-0.169371,-0.070361,0.010390,0.001381,0.044006
155,0.873757,0.813503,0.114925,-0.489674,1.162028,-0.147347,-0.106631,0.047901,-0.036843,0.047744


In [6]:
New_df2['Year'] = np.arange(1850,2006)
final_temp_anomalies_df= df_final.merge(New_df2,on="Year",how='left')
final_temp_anomalies_df.head()

,Year,Glob,NHem,SHem,All forcings,Human,Natural,Anthropogenic tropospheric aerosol,Greenhouse gases,Land use,Orbital changes,Ozone,Solar,Volcanic
0,1880,0.04,-0.27,-0.04,0.013107,0.010159,-0.027790,-0.127200,0.015983,-0.032528,-0.056693,-0.089613,-0.090798,-0.049216
1,1881,0.13,-0.16,0.01,0.063577,0.057503,0.012034,-0.062515,0.002324,-0.085444,-0.082822,-0.094296,-0.072818,-0.019145
2,1882,0.10,-0.20,0.00,0.039591,-0.027156,0.002750,-0.069622,-0.037102,-0.051923,-0.065116,-0.103527,-0.042130,-0.021983
3,1883,0.04,-0.26,-0.06,-0.015938,0.003433,-0.104316,-0.040968,0.010191,-0.032395,-0.066559,-0.090971,-0.054658,-0.087345
4,1884,-0.08,-0.42,-0.14,-0.342759,0.072729,-0.412569,-0.116880,-0.002025,-0.049730,-0.045884,-0.094009,-0.015754,-0.419613


In [7]:
anomalies_df = final_temp_anomalies_df[['Year','Glob','All forcings','Human', 'Natural',
       'Anthropogenic tropospheric aerosol',
       'Greenhouse gases', 'Land use','Orbital changes', 'Ozone', 'Solar',
       'Volcanic']].rename({'Glob':"global_anom"},axis=1)
anomalies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141 entries, 0 to 140
Data columns (total 12 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Year                                141 non-null    int64  
 1   global_anom                         141 non-null    float64
 2   All forcings                        126 non-null    float64
 3   Human                               126 non-null    float64
 4   Natural                             126 non-null    float64
 5   Anthropogenic tropospheric aerosol  126 non-null    float64
 6   Greenhouse gases                    126 non-null    float64
 7   Land use                            126 non-null    float64
 8   Orbital changes                     126 non-null    float64
 9   Ozone                               126 non-null    float64
 10  Solar                               126 non-null    float64
 11  Volcanic                            126 non-n

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
model_df = anomalies_df.dropna()
X = model_df["global_anom"].to_numpy().reshape(-1,1)
y = model_df["All forcings"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
model = LinearRegression()
model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

0.7470364518623644
0.8089932502394163


In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
columns = ['All forcings']
to_predict = pd.DataFrame({"global_anom": anomalies_df[anomalies_df["Year"].isin(
        range(2006,2021,1))]["global_anom"]})
pred_df = pd.DataFrame({"Year": np.arange(2006,2021,1),\
        "global_anom": anomalies_df[anomalies_df["Year"].isin(
            range(2006,2021,1))]["global_anom"]})
for col in columns:
    model_df = anomalies_df.dropna()
    X = model_df["global_anom"].to_numpy().reshape(-1,1)
    y = model_df[col]
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    model = LinearRegression()
    model.fit(X_train,y_train)
    print("###################")
    print(col)
    print(model.score(X_train,y_train))
    print(model.score(X_test,y_test))
    print("###################")
    pred_df[col] = model.predict(to_predict)

###################
All forcings
0.757472020495568
0.809624269681728
###################


In [10]:
columns = [ 'Human']
to_predict2 = pred_df[['global_anom', 'All forcings']] 
for col in columns:
    model_df = anomalies_df.dropna()
    X = model_df[["global_anom",'All forcings']]
    y = model_df[col]
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    model = LinearRegression()
    model.fit(X_train,y_train)
    print("###################")
    print(col)
    print(model.score(X_train,y_train))
    print(model.score(X_test,y_test))
    print("###################")
    pred_df[col] = model.predict(to_predict2)

###################
Human
0.847643879831293
0.8559441047256586
###################


In [13]:
columns = [ 'Greenhouse gases']
to_predict2 = pred_df[['global_anom', 'All forcings']] 
for col in columns:
    model_df = anomalies_df.dropna()
    X = model_df[["global_anom",'All forcings']]
    y = model_df[col]
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    model = LinearRegression()
    model.fit(X_train,y_train)
    print("###################")
    print(col)
    print(model.score(X_train,y_train))
    print(model.score(X_test,y_test))
    print("###################")
    pred_df[col] = model.predict(to_predict2)

###################
Greenhouse gases
0.8749392710049793
0.82091654286823
###################


In [14]:
columns = ['Anthropogenic tropospheric aerosol']
to_predict2 = pred_df[['global_anom', 'All forcings','Greenhouse gases']] 
for col in columns:
    model_df = anomalies_df.dropna()
    X = model_df[["global_anom",'All forcings','Greenhouse gases']]
    y = model_df[col]
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    model = LinearRegression()
    model.fit(X_train,y_train)
    print("###################")
    print(col)
    print(model.score(X_train,y_train))
    print(model.score(X_test,y_test))
    print("###################")
    pred_df[col] = model.predict(to_predict2)

###################
Anthropogenic tropospheric aerosol
0.9491790183816202
0.926546236736091
###################


In [16]:
columns = ['Land use']
to_predict2 = pred_df[['All forcings','Greenhouse gases',
'Anthropogenic tropospheric aerosol']] 
for col in columns:
    model_df = anomalies_df.dropna()
    X = model_df[['All forcings','Greenhouse gases',
    'Anthropogenic tropospheric aerosol']]
    y = model_df[col]
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    model = LinearRegression()
    model.fit(X_train,y_train)
    print("###################")
    print(col)
    print(model.score(X_train,y_train))
    print(model.score(X_test,y_test))
    print("###################")
    pred_df[col] = model.predict(to_predict2)

###################
Land use
0.6363530438191742
0.6747482300368169
###################


In [17]:
pred_df

,Year,global_anom,All forcings,Human,Greenhouse gases,Anthropogenic tropospheric aerosol,Land use
126,2006,0.84,0.808608,0.736079,1.139712,-0.528988,-0.183551
127,2007,0.87,0.834343,0.758369,1.173816,-0.543257,-0.187827
128,2008,0.75,0.731404,0.669210,1.037401,-0.486182,-0.170721
129,2009,0.86,0.825764,0.750939,1.162448,-0.538500,-0.186402
130,2010,0.92,0.877234,0.795518,1.230655,-0.567038,-0.194955
131,2011,0.81,0.782873,0.713789,1.105609,-0.514719,-0.179274
132,2012,0.85,0.817186,0.743509,1.151080,-0.533744,-0.184976
133,2013,0.88,0.842921,0.765798,1.185184,-0.548013,-0.189253
134,2014,0.95,0.902968,0.817808,1.264759,-0.581307,-0.199231
135,2015,1.10,1.031642,0.929256,1.435277,-0.652651,-0.220614


In [18]:
pred_df['Natural'] = pred_df['All forcings'] - pred_df['Human']
columns = ['Orbital changes','Ozone','Solar','Volcanic']
to_predict2 = pred_df[['All forcings','Natural']] 
for col in columns:
    model_df = anomalies_df.dropna()
    X = model_df[['All forcings','Natural']]
    y = model_df[col]
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    model = LinearRegression()
    model.fit(X_train,y_train)
    print("###################")
    print(col)
    print(model.score(X_train,y_train))
    print(model.score(X_test,y_test))
    print("###################")
    pred_df[col] = model.predict(to_predict2)

###################
Orbital changes
0.14616760687965757
-0.0020467876533150076
###################
###################
Ozone
0.5707027643144765
0.5710839333098074
###################
###################
Solar
0.11202388407150843
0.21641345433284687
###################
###################
Volcanic
0.7865801690930319
0.6623440842166413
###################


In [19]:
columns = ['Orbital changes','Ozone','Solar']
for col in columns:
    df5 = anomalies_df[anomalies_df["Year"].\
        isin(range(1990,2005))].dropna()
    df5[f'{col}_percent'] = df5[col]/df5["Natural"]
    print('#################')
    print(col)
    print(df5[f'{col}_percent'].mean())

#################
Orbital changes
0.40664074547239437
#################
Ozone
-0.4240691615863461
#################
Solar
0.3277444301984533


In [20]:
columns = ['Orbital changes','Ozone','Solar']
for col in columns:
    if col == 'Orbital changes':
        pred_df[col] = pred_df['Natural']*-0.3516818676640785
    if col == 'Ozone':
        pred_df[col] = pred_df['Natural']*0.37095615338116783
    pred_df[col] = pred_df['Natural']*0.27210830497222433
pred_df.head()  

,Year,global_anom,All forcings,Human,Greenhouse gases,Anthropogenic tropospheric aerosol,Land use,Natural,Orbital changes,Ozone,Solar,Volcanic
126,2006,0.84,0.808608,0.736079,1.139712,-0.528988,-0.183551,0.072529,0.019736,0.019736,0.019736,0.022612
127,2007,0.87,0.834343,0.758369,1.173816,-0.543257,-0.187827,0.075974,0.020673,0.020673,0.020673,0.024641
128,2008,0.75,0.731404,0.669210,1.037401,-0.486182,-0.170721,0.062194,0.016924,0.016924,0.016924,0.016528
129,2009,0.86,0.825764,0.750939,1.162448,-0.538500,-0.186402,0.074826,0.020361,0.020361,0.020361,0.023964
130,2010,0.92,0.877234,0.795518,1.230655,-0.567038,-0.194955,0.081716,0.022236,0.022236,0.022236,0.028021


In [27]:
anomalies_df.iloc[-15:,2] = pred_df["All forcings"]
anomalies_df.iloc[-15:,3] = pred_df["Human"]
anomalies_df.iloc[-15:,4] = pred_df["Natural"]
anomalies_df.iloc[-15:,5] = pred_df["Anthropogenic tropospheric aerosol"]
anomalies_df.iloc[-15:,6] = pred_df["Greenhouse gases"]
anomalies_df.iloc[-15:,7] = pred_df["Land use"]
anomalies_df.iloc[-15:,8] = pred_df["Orbital changes"]
anomalies_df.iloc[-15:,9] = pred_df["Ozone"]
anomalies_df.iloc[-15:,10] = pred_df["Solar"]
anomalies_df.iloc[-15:,11] = pred_df["Volcanic"]

In [28]:
anomalies_df.to_csv('cleaned/global_anomalies.csv',index=False)